# 131 Final Project - Kickstarter Investigation 

## Set Up

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from collections import Counter

## Read and Investigate the data

In [2]:
ks = pd.read_csv("kickstarter.csv")
ks.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [3]:
ks.describe()

,ID,goal,pledged,backers,usd pledged,usd_pledged_real,usd_goal_real
count,3.786610e+05,3.786610e+05,3.786610e+05,378661.000000,3.748640e+05,3.786610e+05,3.786610e+05
mean,1.074731e+09,4.908079e+04,9.682979e+03,105.617476,7.036729e+03,9.058924e+03,4.545440e+04
std,6.190862e+08,1.183391e+06,9.563601e+04,907.185035,7.863975e+04,9.097334e+04,1.152950e+06
min,5.971000e+03,1.000000e-02,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e-02
25%,5.382635e+08,2.000000e+03,3.000000e+01,2.000000,1.698000e+01,3.100000e+01,2.000000e+03
50%,1.075276e+09,5.200000e+03,6.200000e+02,12.000000,3.947200e+02,6.243300e+02,5.500000e+03
75%,1.610149e+09,1.600000e+04,4.076000e+03,56.000000,3.034090e+03,4.050000e+03,1.550000e+04
max,2.147476e+09,1.000000e+08,2.033899e+07,219382.000000,2.033899e+07,2.033899e+07,1.663614e+08


In [4]:
ks.info()

# Predict: state 

# Prediction Variables: Name / Category / Main_category / Currency / 
#                       deadline / goal / launched / country 

# Variables to make: length_of_campaign / title_score / goal_amount_tier / launch_time_of_day /
#                    launch_year / launch_month / launch_day 

# Descriptive Variables: pledged / backers / usd pledged / usd_pledged_real / usd_goal_real

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378661 entries, 0 to 378660
Data columns (total 15 columns):
ID                  378661 non-null int64
name                378657 non-null object
category            378661 non-null object
main_category       378661 non-null object
currency            378661 non-null object
deadline            378661 non-null object
goal                378661 non-null float64
launched            378661 non-null object
pledged             378661 non-null float64
state               378661 non-null object
backers             378661 non-null int64
country             378661 non-null object
usd pledged         374864 non-null float64
usd_pledged_real    378661 non-null float64
usd_goal_real       378661 non-null float64
dtypes: float64(5), int64(2), object(8)
memory usage: 43.3+ MB


In [5]:
# Categoires and counts for them 
count_categories = ks.category.value_counts()
count_categories


Product Design       22314
Documentary          16139
Music                15727
Tabletop Games       14180
Shorts               12357
Video Games          11830
Food                 11493
Film & Video         10108
Fiction               9169
Fashion               8554
Nonfiction            8318
Art                   8253
Apparel               7166
Theater               7057
Technology            6930
Rock                  6758
Children's Books      6756
Apps                  6345
Publishing            6018
Webseries             5762
Photography           5752
Indie Rock            5657
Narrative Film        5188
Web                   5153
Comics                4996
Crafts                4664
Country & Folk        4451
Design                4199
Hip-Hop               3912
Hardware              3663
                     ...  
Couture                275
Blues                  268
Animals                255
Fabrication Tools      250
Printing               238
Makerspaces            238
M

In [6]:
ks.category.unique() 
ks.category.nunique() #159 unique 

159

In [7]:
ks.main_category.unique()

array(['Publishing', 'Film & Video', 'Music', 'Food', 'Design', 'Crafts',
       'Games', 'Comics', 'Fashion', 'Theater', 'Art', 'Photography',
       'Technology', 'Dance', 'Journalism'], dtype=object)

In [8]:
ks.main_category.nunique()

15

In [9]:
# main_category counts
ks.main_category.value_counts()



Film & Video    63585
Music           51918
Publishing      39874
Games           35231
Technology      32569
Design          30070
Art             28153
Food            24602
Fashion         22816
Theater         10913
Comics          10819
Photography     10779
Crafts           8809
Journalism       4755
Dance            3768
Name: main_category, dtype: int64

In [10]:
#currency
count_currency = ks.currency.value_counts()
count_currency

USD    295365
GBP     34132
EUR     17405
CAD     14962
AUD      7950
SEK      1788
MXN      1752
NZD      1475
DKK      1129
CHF       768
NOK       722
HKD       618
SGD       555
JPY        40
Name: currency, dtype: int64

In [11]:
# Categoires and counts for them 
count_country = ks.country.value_counts()
count_country

US      292627
GB       33672
CA       14756
AU        7839
DE        4171
N,0"      3797
FR        2939
IT        2878
NL        2868
ES        2276
SE        1757
MX        1752
NZ        1447
DK        1113
IE         811
CH         761
NO         708
HK         618
BE         617
AT         597
SG         555
LU          62
JP          40
Name: country, dtype: int64

In [12]:
US_ks = ks[ks['country'] == "US"]
US_ks.groupby(["state"])["goal", "pledged","backers"].mean()

,goal,pledged,backers
state,,,
canceled,76293.964894,2317.801194,25.820670
failed,60664.242108,1331.173917,16.772118
live,84039.947701,6352.720575,77.495402
successful,9695.670955,23212.887104,270.177952
suspended,242796.605263,11119.731497,117.960526


In [13]:
US_ks.groupby(["state"])["goal", "pledged","backers"].median()

,goal,pledged,backers
state,,,
canceled,10000.0,100.0,3
failed,7500.0,106.0,3
live,6500.0,220.0,4
successful,4000.0,5185.0,71
suspended,5000.0,30.5,2


In [14]:
ks.state.value_counts()

failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: state, dtype: int64

In [15]:
# US Success rate 
ks_state = pd.crosstab(index=ks["state"],  # Make a crosstab
                              columns="count")      # Name the count column
ks_state
ks_state/ks_state.sum()
# Around 52% of kickstartes fail, 37% are successful, 10% are canceled, 
# and about 1% are live right now or suspended for the time being

col_0,count
state,
canceled,0.102411
failed,0.522153
live,0.007392
successful,0.353762
suspended,0.004875
undefined,0.009407


In [16]:
# US Success rate 
ks_state_us = pd.crosstab(index=US_ks["state"],  # Make a crosstab
                              columns="count")      # Name the count column
ks_state_us
ks_state_us/ks_state_us.sum()
# In the US, around 52% of kickstartes fail, 37% are successful, 10% are canceled, 
# and about 1% are live right now or suspended for the time being


col_0,count
state,
canceled,0.096748
failed,0.519641
live,0.005946
successful,0.373510
suspended,0.004155


# Initial Investigations 

## How does the timeline influence funding? When does most of the money flow in? 
    

In [17]:

#ks.deadline 
#ks.launched.str
pd.to_datetime(ks.launched).diff(ks.deadline)
#ks.launched.to_string()
#ks.launched, ks.launch_time = 
#ks.launched

duration = datetime.combine(date.min, ks.deadline) - datetime.combine(date.min, ks.launched)


#ks.launched
#dt.datetime(ks.deadline)




TypeError: cannot convert the series to <class 'int'>

## What variables are the best at prediciting if a kickstarted will succeed or not?
   
   ### How accurate can we get with our prediction
    

## Which regions have the best success with kickstarters

In [ ]:
country_states = pd.DataFrame(ks.groupby(["country","state"]).state.count())
sum_of_states_per_country = country_states.groupby(["country"]).state.sum()
country_state_rates = pd.DataFrame(country_states.state / sum_of_states_per_country)
country_state_rates = country_state_rates.unstack()
country_state_rates

In [ ]:
country_state_rates.state.successful.sort_values(ascending=False)
    # is GDP an influencer of success of kickstarters 
country_state_rates.state.failed.sort_values(ascending=False)
    # failure rates vary from 67% to 40% 
    # why is japan so low ? 


## What type of projects succeed the most / fail the most on kickstarter?

In [ ]:
category_states = pd.DataFrame(ks.groupby(["main_category","state"]).state.count())
category_states
sum_of_states_per_category = category_states.groupby(["main_category"]).state.sum()
sum_of_states_per_category
category_state_rates = pd.DataFrame(category_states.state / sum_of_states_per_category)
category_state_rates = category_state_rates.unstack()
category_state_rates

In [ ]:
category_state_rates.state.successful.sort_values(ascending=False)
    # Dance, theater, and comics often do well
    # Tech, Journalism, and Crafts don't do as well
category_state_rates.state.failed.sort_values(ascending=False)
    # Journalism , Food, and Crafts do bad
sum_of_states_per_category.sort_values(ascending=False)
    # Film & Video , music, publishing, and games and tech are popular and more succesptible to failure rates 

## Can your project name influence your success? (length/ contents / etc. ) 

In [ ]:
# Name Length 
ks["name_length"] = ks['name'].str.len()
ks["name_length"].mean()
ks["name_length"].median()

pd.DataFrame(ks.groupby(["state"]).name_length.mean())

# Word Count 
title_words = ks['name'].str.split()
ks["title_word_count"] = title_words.str.len()
ks["title_word_count"].mean()
ks["title_word_count"].median()
pd.DataFrame(ks.groupby(["state"]).title_word_count.mean())

#title_words

# Text Analysis 





## What sort of funding scheme makes for a successful kickstarter (small amounts from many or large amounts from a few)
    #### Number of backers influence your success 
   

In [ ]:
ks["average_backed_amount"] = (ks.pledged / ks.backers)
ks["pledge_goal_diff"] = (ks.pledged - ks.goal) # by how much are failing kickstarters missing their goals 

ks.groupby(["state"])["goal", "pledged","backers","average_backed_amount","pledge_goal_diff"].mean()
ks.groupby(["state"])["goal", "pledged","backers","average_backed_amount","pledge_goal_diff"].median()
    # successful kickstarters have...
        # MANY more backers than any other 
        # smaller goal than failed brojects by $3500 

#success_ks = ks[ks.state == "successful"]
#success_ks




## Need plots of stuff for distributions and better understanding the data

